In [1]:
#importing necessary libararies

import numpy as np
from scipy.stats import ks_2samp
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd


Part 1 :A/B Testing using Ad Click Prediction

In [2]:

# printing first few rows of dataset
df = pd.read_csv("ad_click_dataset.csv")
print(df.head())

     id full_name   age      gender device_type ad_position browsing_history  \
0   670   User670  22.0         NaN     Desktop         Top         Shopping   
1  3044  User3044   NaN        Male     Desktop         Top              NaN   
2  5912  User5912  41.0  Non-Binary         NaN        Side        Education   
3  5418  User5418  34.0        Male         NaN         NaN    Entertainment   
4  9452  User9452  39.0  Non-Binary         NaN         NaN     Social Media   

  time_of_day  click  
0   Afternoon      1  
1         NaN      1  
2       Night      1  
3     Evening      1  
4     Morning      0  


In [3]:
print(df.info())   #getting information of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                10000 non-null  int64  
 1   full_name         10000 non-null  object 
 2   age               5234 non-null   float64
 3   gender            5307 non-null   object 
 4   device_type       8000 non-null   object 
 5   ad_position       8000 non-null   object 
 6   browsing_history  5218 non-null   object 
 7   time_of_day       8000 non-null   object 
 8   click             10000 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 703.3+ KB
None


In [4]:

# doing data cleaning
print("\nMissing values:\n", df.isnull().sum())

df = df.dropna(subset=['click'])
df = df.dropna(subset=['ad_position'])


Missing values:
 id                     0
full_name              0
age                 4766
gender              4693
device_type         2000
ad_position         2000
browsing_history    4782
time_of_day         2000
click                  0
dtype: int64


In [5]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8000 non-null   int64  
 1   full_name         8000 non-null   object 
 2   age               4186 non-null   float64
 3   gender            4221 non-null   object 
 4   device_type       6433 non-null   object 
 5   ad_position       8000 non-null   object 
 6   browsing_history  4227 non-null   object 
 7   time_of_day       6408 non-null   object 
 8   click             8000 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 625.0+ KB
None


In [6]:
df


,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
5,5942,User5942,NaN,Non-Binary,NaN,Bottom,Social Media,Evening,1
6,7808,User7808,26.0,Female,Desktop,Top,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
9992,5818,User5818,NaN,NaN,Tablet,Top,Social Media,Night,1
9995,8510,User8510,NaN,NaN,Mobile,Top,Education,NaN,0
9996,7843,User7843,NaN,Female,Desktop,Bottom,Entertainment,NaN,0
9997,3914,User3914,NaN,Male,Mobile,Side,NaN,Morning,0


In [7]:
print(df['gender'].unique())
print(df['ad_position'].unique())


[nan 'Male' 'Non-Binary' 'Female']
['Top' 'Side' 'Bottom']


In [8]:
# The categorical columns are gender,ad_position
df['gender'] = df['gender'].astype('category')
df['ad_position'] = df['ad_position'].astype('category')

In [9]:
# converting to categorical
df['gender'] = df['gender'].astype('category')
df['ad_position'] = df['ad_position'].astype('category')

# map gender categories
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1, 'Non-Binary': 2})

# convert to category again and add -1 to handle NaN values
df['gender'] = df['gender'].astype('category')
df['gender'] = df['gender'].cat.add_categories([-1])
df['gender'] = df['gender'].fillna(-1)

# mapping ad_position categories
df['ad_position'] = df['ad_position'].map({'Top':0,'Bottom':1,'Side':2})


In [10]:
df.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,-1,Desktop,0,Shopping,Afternoon,1
1,3044,User3044,NaN,0,Desktop,0,NaN,NaN,1
2,5912,User5912,41.0,2,NaN,2,Education,Night,1
5,5942,User5942,NaN,2,NaN,1,Social Media,Evening,1
6,7808,User7808,26.0,1,Desktop,0,NaN,NaN,1


In [11]:
# splitting into group A (Top) and group B (bottom)
group_a = df[df['ad_position'] == 0]
group_b = df[df['ad_position'] == 1]

In [12]:
group_a.head()


,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,-1,Desktop,0,Shopping,Afternoon,1
1,3044,User3044,NaN,0,Desktop,0,NaN,NaN,1
6,7808,User7808,26.0,1,Desktop,0,NaN,NaN,1
15,7529,User7529,NaN,-1,NaN,0,Entertainment,Afternoon,0
18,2124,User2124,NaN,0,Desktop,0,NaN,Evening,1


In [13]:
group_b.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
5,5942,User5942,NaN,2,NaN,1,Social Media,Evening,1
8,7993,User7993,NaN,2,Mobile,1,Social Media,NaN,1
9,4509,User4509,NaN,-1,NaN,1,Education,Afternoon,1
10,2595,User2595,NaN,-1,NaN,1,NaN,Morning,1
11,7466,User7466,47.0,-1,Mobile,1,NaN,Afternoon,1


In [14]:
# clicks and impressions
clicks_A = group_a['click'].sum()
clicks_B = group_b['click'].sum()
n_A = len(group_a)
n_B = len(group_b)

In [15]:
# CTRs :Click through rate
ctr_A = clicks_A / n_A
ctr_B = clicks_B / n_B

print(f"CTR (Top ads): {ctr_A:.4f}")
print(f"CTR (Bottom ads): {ctr_B:.4f}")

CTR (Top ads): 0.6350
CTR (Bottom ads): 0.6873


In [16]:
# performing z-test
z_score, p_value = proportions_ztest([clicks_A, clicks_B], [n_A, n_B])
print(f"Z_score: {z_score:.4f}")
print(f"P_value: {p_value:.4f}")

Z_score: -4.0642
P_value: 0.0000


In [17]:
alpha = 0.05
if p_value < alpha:
    print("Result: Statistically significant difference in CTR between top and bottom positions.")
else:
    print("Result: No statistically significant difference in CTR.")

Result: Statistically significant difference in CTR between top and bottom positions.


Part2 :  Covariate Shift Detection Using Air Quality Data

In [18]:
# loading air quality datasets
train = pd.read_csv("train.csv")
test1 = pd.read_csv("test1.csv")
test2 = pd.read_csv("test2.csv")

In [19]:
train.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,1849,26/05/2004,19.00.00,-200,1130.0,-200.0,"22,7",1368.0,-200.0,933.0,-200.0,1709.0,1269.0,"26,7","19,5","0,6754",NaN,NaN
1,2533,24/06/2004,07.00.00,"1,2",1030.0,-200.0,"6,9",851.0,102.0,824.0,68.0,1700.0,983.0,"21,9","57,0","1,4742",NaN,NaN
2,3047,15/07/2004,17.00.00,"3,2",1164.0,-200.0,"20,3",1306.0,259.0,648.0,198.0,1886.0,1218.0,"35,5","19,1","1,0888",NaN,NaN
3,805,13/04/2004,07.00.00,"3,9",1496.0,524.0,"19,1",1272.0,328.0,667.0,130.0,2011.0,1399.0,"11,0","64,2","0,8398",NaN,NaN
4,2962,12/07/2004,04.00.00,-200,780.0,-200.0,"1,8",568.0,24.0,1200.0,34.0,1331.0,501.0,"19,9","51,3","1,1803",NaN,NaN


In [20]:
test1.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,3123,18/07/2004,21.00.00,"1,2",1067.0,-200.0,"9,0",938.0,102.0,825.0,99.0,1520.0,912.0,"29,7","24,8","1,0160",NaN,NaN
1,877,16/04/2004,07.00.00,"4,5",1657.0,523.0,"23,2",1384.0,352.0,579.0,109.0,2176.0,1600.0,"12,8","71,0","1,0428",NaN,NaN
2,3457,01/08/2004,19.00.00,"1,4",1037.0,-200.0,"8,0",900.0,75.0,817.0,95.0,1584.0,619.0,"33,1","32,7","1,6200",NaN,NaN
3,1494,12/05/2004,00.00.00,"1,7",1122.0,-200.0,"8,7",926.0,105.0,805.0,88.0,1619.0,1174.0,"16,9","58,8","1,1250",NaN,NaN
4,713,09/04/2004,11.00.00,"2,6",-200.0,262.0,"-200,0",-200.0,219.0,-200.0,121.0,-200.0,-200.0,-200,-200,-200,NaN,NaN


In [21]:
test2.head()

,Unnamed: 0,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,8500,27/02/2005,22.00.00,"1,0",875.0,-200.0,"2,1",594.0,128.0,1079.0,105.0,793.0,451.0,"4,5","48,0","0,4085",NaN,NaN
1,8501,27/02/2005,23.00.00,"1,3",943.0,-200.0,"3,9",703.0,169.0,950.0,119.0,870.0,581.0,"4,3","48,6","0,4069",NaN,NaN
2,8502,28/02/2005,00.00.00,"1,6",947.0,-200.0,"3,8",697.0,215.0,913.0,150.0,878.0,698.0,"4,0","50,0","0,4115",NaN,NaN
3,8503,28/02/2005,01.00.00,"1,0",865.0,-200.0,"1,8",566.0,111.0,1119.0,94.0,797.0,423.0,"4,0","52,9","0,4338",NaN,NaN
4,8504,28/02/2005,02.00.00,"0,6",823.0,-200.0,"1,0",503.0,60.0,1268.0,56.0,755.0,332.0,"4,0","51,0","0,4200",NaN,NaN


In [29]:
train_no2 = train['NO2(GT)']
test1_no2 = test1['NO2(GT)']
test2_no2 = test2['NO2(GT)']

 #removing the places where the value of NO2(GT) column is negative
 
train_no2_new = train_no2[(train_no2 >= 0)].dropna()
test1_no2_new = test1_no2[(test1_no2 >= 0)].dropna()
test2_no2_new = test2_no2[(test2_no2 >= 0)].dropna()

In [30]:
print("\nTrain NO2(GT):", train_no2.describe())
print("Test1 NO2(GT):", test1_no2.describe())
print("Test2 NO2(GT):", test2_no2.describe())


Train NO2(GT): count    3200.000000
mean       45.605625
std       114.663990
min      -200.000000
25%        47.750000
50%        84.000000
75%       114.000000
max       233.000000
Name: NO2(GT), dtype: float64
Test1 NO2(GT): count    800.000000
mean      42.621250
std      117.115831
min     -200.000000
25%       46.750000
50%       84.000000
75%      114.000000
max      223.000000
Name: NO2(GT), dtype: float64
Test2 NO2(GT): count    800.000000
mean     129.682500
std       61.071957
min     -200.000000
25%      100.000000
50%      133.000000
75%      163.250000
max      248.000000
Name: NO2(GT), dtype: float64


In [31]:
# just preview
print("\nTrain NO2(GT):", train_no2_new.describe())
print("Test1 NO2(GT):", test1_no2_new.describe())
print("Test2 NO2(GT):", test2_no2_new.describe())


Train NO2(GT): count    2668.000000
mean       94.579460
std        36.584146
min         5.000000
25%        67.000000
50%        94.000000
75%       120.000000
max       233.000000
Name: NO2(GT), dtype: float64
Test1 NO2(GT): count    659.000000
mean      94.532625
std       36.639541
min        5.000000
25%       66.000000
50%       94.000000
75%      118.000000
max      223.000000
Name: NO2(GT), dtype: float64
Test2 NO2(GT): count    788.000000
mean     134.703046
std       45.870772
min       25.000000
25%      101.000000
50%      134.000000
75%      164.000000
max      248.000000
Name: NO2(GT), dtype: float64


In [32]:
# comparing train vs test1
ks_stat1, p_val1 = ks_2samp(train_no2_new, test1_no2_new)
print(f"Test1 vs. Train: KS Stat = {ks_stat1:.4f}, P-Value = {p_val1:.4f}")


# comparing train vs test2
ks_stat2, p_val2 = ks_2samp(train_no2_new, test2_no2_new)
print(f"Test2 vs. Train: KS Stat = {ks_stat2:.4f}, P-Value = {p_val2:.4f}")



Test1 vs. Train: KS Stat = 0.0171, P-Value = 0.9971
Test2 vs. Train: KS Stat = 0.3689, P-Value = 0.0000


In [33]:
alpha = 0.05
shift_test1 = p_val1 < alpha  # True if shift detected
shift_test2 = p_val2 < alpha

print(f"Covariate Shift in Test1: {shift_test1}")
print(f"Covariate Shift in Test2: {shift_test2}")

Covariate Shift in Test1: False
Covariate Shift in Test2: True


In [26]:
# Covariance shift

if ks_stat1 > ks_stat2:
    print("Test1 shows a larger covariate shift from training data.")
else:
    print("Test2 shows a larger covariate shift from training data.")


Test2 shows a larger covariate shift from training data.
